#Samer Baslan
#Perceiver Image Classification Homework: MNIST dataset
#CMPE-297: Special Topics Spring 2021

Resource: https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/vision/ipynb/perceiver_image_classification.ipynb#scrollTo=gZmhMWAPFf5b

##Setup

In [1]:
!pip install -U tensorflow-addons

     |████████████████████████████████| 1.1 MB 6.8 MB/s 


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Oct  7 04:15:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##Data Prepation

In [4]:
num_classes = 10
input_shape = (28, 28, 1)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

y_train = y_train.reshape(y_train.shape[0], 1)
y_test = y_test.reshape(y_test.shape[0], 1)


print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

11501568/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1) - y_train shape: (60000, 1)
x_test shape: (10000, 28, 28, 1) - y_test shape: (10000, 1)


##Hyperparameter Configuration

In [5]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 64
num_epochs = 10
dropout_rate = 0.2
image_size = 56 #Resize input images to this size
patch_size = 2 #Size of the patches to be extracted from the input images
               #To use each pixel as individual input in the data array, set to 1
num_patches = (image_size // patch_size) ** 2 #Size of the data array
latent_dim = 256 #Size of latent array
projection_dim = 256 #Embedding size of each element in the data and latent array
num_heads = 8 #Number of Transformer heads
num_transformer_blocks = 4
num_iterations = 2 #Repetitions of the cross-attention and Transformer modules

ffn_units = [projection_dim, projection_dim,] #Size of the Transformer Feedforward network
classifier_units = [
                    projection_dim, 
                    num_classes,
                    ] #Size of the Feedforward network of the final classifier

print(f"Image size: {image_size} X {image_size} = {image_size ** 2}")
print(f"Patch size: {patch_size} X {patch_size} = {patch_size ** 2}")
print(f"Patches per image: {num_patches}")
print(f"Elements per patch (3 channels): {(patch_size ** 2) * 3}")
print(f"Latent array shape: {latent_dim} X {projection_dim}")
print(f"Data array shape: {num_patches} X {projection_dim}")

Image size: 56 X 56 = 3136
Patch size: 2 X 2 = 4
Patches per image: 784
Elements per patch (3 channels): 12
Latent array shape: 256 X 256
Data array shape: 784 X 256


##Data Augmentation

In [6]:
data_augmentation = keras.Sequential(
    [
     layers.Normalization(),
     layers.Resizing(image_size, image_size),
     layers.RandomFlip("horizontal"),
     layers.RandomZoom(
         height_factor = 0.2, width_factor = 0.2
     ),
    ],
    name = "data_augmentation",
)

#Compute the mean and the vavriance of the training data for normalization
data_augmentation.layers[0].adapt(x_train)

##Feed Forward Network (FFN)

In [7]:
def create_ffn(hidden_units, dropout_rate):
  ffn_layers = []
  for units in hidden_units[:-1]:
    ffn_layers.append(layers.Dense(units, activation = tf.nn.gelu))

  ffn_layers.append(layers.Dense(units = hidden_units[-1]))
  ffn_layers.append(layers.Dropout(dropout_rate))

  ffn = keras.Sequential(ffn_layers)
  return ffn

##Implement patch creation as a layer

In [8]:
class Patches(layers.Layer):
  def __init__(self, patch_size):
    super(Patches, self).__init__()
    self.patch_size = patch_size

  def call(self, images):
    batch_size = tf.shape(images)[0]
    patches = tf.image.extract_patches(
        images = images,
        sizes = [1, self.patch_size, self.patch_size, 1],
        strides = [1, self.patch_size, self.patch_size, 1],
        rates = [1, 1, 1, 1],
        padding = "VALID",
    )
    patch_dims = patches.shape[-1]
    patches = tf.reshape(patches, [batch_size, -1, patch_dims])
    return patches

##Implement the patch encoding layer

The `PatchEncoder` layer will linearly transform a patch by projecting it into a vector of size `latent_dim`. In addition, it adds a learnable position embedding to the projected vector.

In [9]:
class PatchEncoder(layers.Layer):
  def __init__(self, num_patches, projection_dim):
    super(PatchEncoder, self).__init__()
    self.num_patches = num_patches
    self.projection = layers.Dense(units = projection_dim)
    self.position_embedding = layers.Embedding(
        input_dim = num_patches, output_dim = projection_dim
    )

  def call(self, patches):
    positions = tf.range(start = 0, limit = self.num_patches, delta = 1)
    encoded = self.projection(patches) + self.position_embedding(positions)
    return encoded

##Build the Perceiver model

The Perceiver consists of two modules:
1. cross-attention module
2. standard Transformer with self attention

###Cross-attention module

In [10]:
def create_cross_attention_module(
    latent_dim, data_dim, projection_dim, ffn_units, droput_rate
    ):
  
  inputs = {
      #[1, latent_dim, projection_dim]
      "latent_array": layers.Input(shape = (latent_dim, projection_dim)),
      #[batch_size, data_dim, projection_dim] 
      "data_array": layers.Input(shape = (data_dim, projection_dim)), 
  }

  #Apply layer norm to the inputs
  latent_array = layers.LayerNormalization(epsilon = 1e-6)(inputs["latent_array"])
  data_array = layers.LayerNormalization(epsilon = 1e-6)(inputs["data_array"])

  #Create query tensor: [1, latent_dim, projection_dim]
  query = layers.Dense(units = projection_dim)(latent_array)

  #Create key tensor: [batch_size, data_dim, projection_dim]
  key = layers.Dense(units = projection_dim)(data_array)

  #Create value tensor: [batch_size, data_dim, projection_dim]
  value = layers.Dense(units = projection_dim)(data_array)

  #Generate cross-attention outputs: [batch_size, latent_dim, projection_dim]
  attention_output = layers.Attention(use_scale = True, dropout = 0.1)(
      [query, key, value], return_attention_scores = False
  )

  #Skip connection to 1
  attention_output = layers.Add()([attention_output, latent_array])

  #Apply layer norm
  attention_output = layers.LayerNormalization(epsilon = 1e-6)(attention_output)

  #Apply Feedforward network
  ffn = create_ffn(ffn_units, dropout_rate)
  outputs = ffn(attention_output)

  #Skip connection to 2
  outputs = layers.Add()([outputs, attention_output])

  #Create the Keras model
  model = keras.Model(inputs = inputs, outputs = outputs)
  return model

###Transformer module

In [11]:
def create_transformer_module(
    latent_dim, 
    projection_dim, 
    num_heads, 
    num_transformer_blocks, 
    ffn_units, 
    dropout_rate,):

  #input shape: [1, latent_dim, projection_dim]
  inputs = layers.Input(shape = (latent_dim, projection_dim))

  x0 = inputs
  #Create multiple layers of the Transformer block
  for _ in range(num_transformer_blocks):
    #Apply layer normalization 1
    x1 = layers.LayerNormalization(epsilon = 1e-6)(x0)
    #Create a multi-head self-attention layer
    attention_output = layers.MultiHeadAttention(
        num_heads = num_heads, key_dim = projection_dim, dropout = 0.1
    )(x1, x1)
    #Skip connection 1
    x2 = layers.Add()([attention_output, x0])
    #Apply layer normalization 2
    x3 = layers.LayerNormalization(epsilon = 1e-6)(x2)
    #Apply feedforward network
    ffn = create_ffn(hidden_units = ffn_units, dropout_rate = dropout_rate)
    x3 = ffn(x3)
    #Skip connection 2
    x0 = layers.Add()([x3, x2])

  #Create the Keras model
  model = keras.Model(inputs = inputs, outputs = x0)
  return model

###Perceiver model

In [12]:
class Perceiver(keras.Model):
    def __init__(
        self,
        patch_size,
        data_dim,
        latent_dim,
        projection_dim,
        num_heads,
        num_transformer_blocks,
        ffn_units,
        dropout_rate,
        num_iterations,
        classifier_units,
    ):
        super(Perceiver, self).__init__()

        self.latent_dim = latent_dim
        self.data_dim = data_dim
        self.patch_size = patch_size
        self.projection_dim = projection_dim
        self.num_heads = num_heads
        self.num_transformer_blocks = num_transformer_blocks
        self.ffn_units = ffn_units
        self.dropout_rate = dropout_rate
        self.num_iterations = num_iterations
        self.classifier_units = classifier_units

    def build(self, input_shape):
        # Create latent array.
        self.latent_array = self.add_weight(
            shape=(self.latent_dim, self.projection_dim),
            initializer="random_normal",
            trainable=True,
        )

        # Create patching module.
        self.patcher = Patches(self.patch_size)

        # Create patch encoder.
        self.patch_encoder = PatchEncoder(self.data_dim, self.projection_dim)

        # Create cross-attenion module.
        self.cross_attention = create_cross_attention_module(
            self.latent_dim,
            self.data_dim,
            self.projection_dim,
            self.ffn_units,
            self.dropout_rate,
        )

        # Create Transformer module.
        self.transformer = create_transformer_module(
            self.latent_dim,
            self.projection_dim,
            self.num_heads,
            self.num_transformer_blocks,
            self.ffn_units,
            self.dropout_rate,
        )

        # Create global average pooling layer.
        self.global_average_pooling = layers.GlobalAveragePooling1D()

        # Create a classification head.
        self.classification_head = create_ffn(
            hidden_units=self.classifier_units, dropout_rate=self.dropout_rate
        )

        super(Perceiver, self).build(input_shape)

    def call(self, inputs):
        # Augment data.
        augmented = data_augmentation(inputs)
        # Create patches.
        patches = self.patcher(augmented)
        # Encode patches.
        encoded_patches = self.patch_encoder(patches)
        # Prepare cross-attention inputs.
        cross_attention_inputs = {
            "latent_array": tf.expand_dims(self.latent_array, 0),
            "data_array": encoded_patches,
        }
        # Apply the cross-attention and the Transformer modules iteratively.
        for _ in range(self.num_iterations):
            # Apply cross-attention from the latent array to the data array.
            latent_array = self.cross_attention(cross_attention_inputs)
            # Apply self-attention Transformer to the latent array.
            latent_array = self.transformer(latent_array)
            # Set the latent array of the next iteration.
            cross_attention_inputs["latent_array"] = latent_array

        # Apply global average pooling to generate a [batch_size, projection_dim] repesentation tensor.
        representation = self.global_average_pooling(latent_array)
        # Generate logits.
        logits = self.classification_head(representation)
        return logits


##Compile, tran, and evaluate model

In [13]:
def run_experiment(model):
  
  #Create LAMB optimizer with weight decay
  optimizer = tfa.optimizers.LAMB(
      learning_rate = learning_rate, weight_decay_rate = weight_decay,
  )

  #Compile the model
  model.compile(
      optimizer = optimizer,
      loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
      metrics = [
          keras.metrics.SparseCategoricalAccuracy(name = "acc"),
          keras.metrics.SparseTopKCategoricalAccuracy(5, name = "top5-acc"),
      ],
  )

  #Create a learning rate scheduler callback
  reduce_lr = keras.callbacks.ReduceLROnPlateau(
      monitor = "val_loss", factor = 0.2, patience = 3
  )

  #Create an early stopping callback
  early_stopping = tf.keras.callbacks.EarlyStopping(
      monitor = "vall_loss", patience = 15, restore_best_weights= True
  )

  #Fit the model
  history = model.fit(
      x = x_train,
      y = y_train,
      batch_size = batch_size,
      epochs = num_epochs,
      validation_split = 0.1,
      callbacks = [early_stopping, reduce_lr],
  )

  _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
  print(f"Test accuracy: {round(accuracy * 100, 2)}%")
  print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")


  #Return history to plot learning curves
  return history

In [14]:
perceiver_classifier = Perceiver(
    patch_size,
    num_patches,
    latent_dim,
    projection_dim,
    num_heads,
    num_transformer_blocks,
    ffn_units,
    dropout_rate,
    num_iterations,
    classifier_units,
)


history = run_experiment(perceiver_classifier)

Epoch 1/10
844/844 [==============================] - 643s 738ms/step - loss: 1.3782 - acc: 0.5072 - top5-acc: 0.9083 - val_loss: 0.5750 - val_acc: 0.7910 - val_top5-acc: 0.9917
Epoch 2/10
844/844 [==============================] - 621s 736ms/step - loss: 0.8768 - acc: 0.6855 - top5-acc: 0.9726 - val_loss: 0.4028 - val_acc: 0.8628 - val_top5-acc: 0.9938
Epoch 3/10
844/844 [==============================] - 621s 736ms/step - loss: 0.7308 - acc: 0.7344 - top5-acc: 0.9823 - val_loss: 0.2939 - val_acc: 0.9010 - val_top5-acc: 0.9963
Epoch 4/10
844/844 [==============================] - 621s 735ms/step - loss: 0.6435 - acc: 0.7609 - top5-acc: 0.9863 - val_loss: 0.3714 - val_acc: 0.8700 - val_top5-acc: 0.9943
Epoch 5/10
844/844 [==============================] - 621s 736ms/step - loss: 0.5545 - acc: 0.7904 - top5-acc: 0.9894 - val_loss: 0.1895 - val_acc: 0.9397 - val_top5-acc: 0.9982
Epoch 6/10
844/844 [==============================] - 621s 736ms/step - loss: 0.4894 - acc: 0.8094 - top5-acc: